In [28]:
# Load libraries
import pandas as pd
import altair as alt
import numpy as np
import statsmodels.api as sm


In [29]:
# Step 1: Load price elasticity from regression analysis
# Re-run the regression to get the elasticity coefficient

price_data = pd.read_csv("Byte_Datasets/average_ticket_price_by_month.csv")
price_data_valid = price_data[price_data["Estimated_Attendees"] > 0].copy()

# Calculate log transformations
price_data_valid["Log_Price"] = np.log(price_data_valid["Average_Ticket_Price"])
price_data_valid["Log_Attendees"] = np.log(price_data_valid["Estimated_Attendees"])

# Run regression: log(Quantity) = β₀ + β₁ × log(Price)
X = price_data_valid["Log_Price"]
y = price_data_valid["Log_Attendees"]
X_with_const = sm.add_constant(X)
model = sm.OLS(y, X_with_const)
results = model.fit()

# Extract elasticity
elasticity = results.params['Log_Price']

print("="*80)
print("PRICE ELASTICITY OF DEMAND")
print("="*80)
print(f"Elasticity Coefficient: {elasticity:.4f}")
print(f"Interpretation: A 1% increase in price leads to a {elasticity:.4f}% change in quantity")
print("="*80)


PRICE ELASTICITY OF DEMAND
Elasticity Coefficient: -3.3891
Interpretation: A 1% increase in price leads to a -3.3891% change in quantity


In [30]:
# Step 2: Load marginal costs per month
marginal_costs = pd.read_csv("Byte_Datasets/marginal_cost_per_attendee.csv")

print("\nMarginal Cost Per Attendee by Month:")
print(marginal_costs[["Month", "Marginal_Cost_Per_Attendee"]])



Marginal Cost Per Attendee by Month:
    Month  Marginal_Cost_Per_Attendee
0       1                 -478.365138
1       2                  -68.903160
2       3                  -70.764587
3       4                  -84.962587
4       5                 -114.354535
5       6                 -159.067118
6       7                 -140.794845
7       8                  -86.703573
8       9                  -95.095865
9      10                  -69.842073
10     11                 -742.826442
11     12                    0.000000


In [31]:
# Step 3: Calculate Optimal Price Using Lerner Index
# Lerner Index formula: (P - MC) / P = 1 / |E|
# Solving for P: P = MC / (1 - 1/|E|)
# Where |E| is the ABSOLUTE VALUE of elasticity

# Use absolute value of elasticity
abs_elasticity = abs(elasticity)

print(f"\nAbsolute Elasticity |E|: {abs_elasticity:.4f}")

# Merge actual prices with marginal costs
pricing_analysis = price_data.merge(
    marginal_costs[["Month", "Marginal_Cost_Per_Attendee"]], 
    on="Month"
)

# Ensure marginal costs are positive (use absolute value)
pricing_analysis["MC_Positive"] = abs(pricing_analysis["Marginal_Cost_Per_Attendee"])

# Calculate optimal price using Lerner Index
# P = MC / (1 - 1/|E|)
pricing_analysis["Optimal_Price"] = pricing_analysis["MC_Positive"] / (1 - (1 / abs_elasticity))

# Calculate markup percentage using positive MC
pricing_analysis["Markup_Percentage"] = ((pricing_analysis["Optimal_Price"] - pricing_analysis["MC_Positive"]) / pricing_analysis["MC_Positive"]) * 100

# Calculate price difference
pricing_analysis["Price_Difference"] = pricing_analysis["Optimal_Price"] - pricing_analysis["Average_Ticket_Price"]
pricing_analysis["Price_Difference_Percentage"] = (pricing_analysis["Price_Difference"] / pricing_analysis["Average_Ticket_Price"]) * 100

print("\n" + "="*80)
print("OPTIMAL PRICING ANALYSIS (LERNER INDEX)")
print("="*80)
print(pricing_analysis[["Month", "MC_Positive", "Average_Ticket_Price", "Optimal_Price", "Price_Difference"]])
print("="*80)



Absolute Elasticity |E|: 3.3891

OPTIMAL PRICING ANALYSIS (LERNER INDEX)
    Month  MC_Positive  Average_Ticket_Price  Optimal_Price  Price_Difference
0       1   478.365138            136.563835     678.595639        542.031804
1       2    68.903160             88.736173      97.744129          9.007956
2       3    70.764587             85.459006     100.384699         14.925693
3       4    84.962587             83.806573     120.525591         36.719018
4       5   114.354535             87.361049     162.220201         74.859152
5       6   159.067118             74.914130     225.648243        150.734113
6       7   140.794845             86.906880     199.727698        112.820817
7       8    86.703573             89.061381     122.995306         33.933925
8       9    95.095865             81.744410     134.900380         53.155970
9      10    69.842073             84.202102      99.076045         14.873944
10     11   742.826442            105.862954    1053.753177        9

In [32]:
# Summary statistics
valid_pricing = pricing_analysis[pricing_analysis["Estimated_Attendees"] > 0]

print("\nSUMMARY STATISTICS:")
print(f"Average Current Price: ${valid_pricing['Average_Ticket_Price'].mean():.2f}")
print(f"Average Optimal Price: ${valid_pricing['Optimal_Price'].mean():.2f}")
print(f"Average Marginal Cost: ${valid_pricing['MC_Positive'].mean():.2f}")
print(f"\nAverage Markup (Optimal): {valid_pricing['Markup_Percentage'].mean():.1f}%")
print(f"\nPrice Adjustment Needed:")
print(f"  Average difference: ${valid_pricing['Price_Difference'].mean():.2f}")
print(f"  Average % change: {valid_pricing['Price_Difference_Percentage'].mean():.1f}%")

# Check if generally overpriced or underpriced
if valid_pricing['Price_Difference'].mean() > 0:
    print(f"\n⚠️  Current prices are on average TOO LOW by ${abs(valid_pricing['Price_Difference'].mean()):.2f}")
    print("     RECOMMENDATION: Increase ticket prices to maximize profit")
else:
    print(f"\n⚠️  Current prices are on average TOO HIGH by ${abs(valid_pricing['Price_Difference'].mean()):.2f}")
    print("     RECOMMENDATION: Decrease ticket prices to maximize profit")



SUMMARY STATISTICS:
Average Current Price: $91.33
Average Optimal Price: $272.32
Average Marginal Cost: $191.97

Average Markup (Optimal): 41.9%

Price Adjustment Needed:
  Average difference: $181.00
  Average % change: 172.8%

⚠️  Current prices are on average TOO LOW by $181.00
     RECOMMENDATION: Increase ticket prices to maximize profit


In [33]:
# Step 4: Visualize Optimal vs Actual Pricing
alt.data_transformers.disable_max_rows()

# Add month labels
month_names = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 
               7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
pricing_analysis["Month_Label"] = pricing_analysis["Month"].map(month_names)

# Filter out months with no attendees
pricing_filtered = pricing_analysis[pricing_analysis["Estimated_Attendees"] > 0].copy()

# Reshape data for grouped bar chart
pricing_long = pd.melt(
    pricing_filtered,
    id_vars=['Month', 'Month_Label', 'MC_Positive'],
    value_vars=['Average_Ticket_Price', 'Optimal_Price'],
    var_name='Price_Type',
    value_name='Price'
)

# Rename for better labels
pricing_long['Price_Type'] = pricing_long['Price_Type'].replace({
    'Average_Ticket_Price': 'Current Price',
    'Optimal_Price': 'Optimal Price (Lerner Index)'
})

# Create grouped bar chart
bars = alt.Chart(pricing_long).mark_bar().encode(
    x=alt.X('Month:O', title='Month', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('Price:Q', title='Ticket Price ($)'),
    color=alt.Color('Price_Type:N', 
                    title='Price Type',
                    scale=alt.Scale(domain=['Current Price', 'Optimal Price (Lerner Index)'],
                                   range=['#3498db', '#2ecc71'])),
    xOffset='Price_Type:N',
    tooltip=[
        alt.Tooltip('Month_Label:N', title='Month'),
        alt.Tooltip('Price_Type:N', title='Type'),
        alt.Tooltip('Price:Q', title='Price', format='$,.2f'),
        alt.Tooltip('MC_Positive:Q', title='Marginal Cost', format='$,.2f')
    ]
).properties(
    title='Current vs Optimal Ticket Prices by Month (Lerner Index)',
    width=700,
    height=400
)

bars


alt.Chart(...)

In [34]:
# Additional visualization: Show all three (MC, Current, Optimal) as lines
line_data = pricing_filtered[['Month', 'Month_Label', 'MC_Positive', 'Average_Ticket_Price', 'Optimal_Price']].copy()

# Reshape for line chart
line_long = pd.melt(
    line_data,
    id_vars=['Month', 'Month_Label'],
    value_vars=['MC_Positive', 'Average_Ticket_Price', 'Optimal_Price'],
    var_name='Metric',
    value_name='Value'
)

# Rename for better labels
line_long['Metric'] = line_long['Metric'].replace({
    'MC_Positive': 'Marginal Cost',
    'Average_Ticket_Price': 'Current Price',
    'Optimal_Price': 'Optimal Price'
})

# Create line chart
line_chart = alt.Chart(line_long).mark_line(strokeWidth=3).encode(
    x=alt.X('Month:O', title='Month', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('Value:Q', title='Price / Cost ($)'),
    color=alt.Color('Metric:N',
                    scale=alt.Scale(domain=['Marginal Cost', 'Current Price', 'Optimal Price'],
                                   range=['#e67e22', '#3498db', '#2ecc71']),
                    legend=alt.Legend(title='Metric'))
).properties(
    title='Pricing Analysis: Marginal Cost, Current Price, and Optimal Price',
    width=700,
    height=400
)

# Add points
points = alt.Chart(line_long).mark_point(size=100, filled=True).encode(
    x=alt.X('Month:O'),
    y=alt.Y('Value:Q'),
    color=alt.Color('Metric:N',
                    scale=alt.Scale(domain=['Marginal Cost', 'Current Price', 'Optimal Price'],
                                   range=['#e67e22', '#3498db', '#2ecc71'])),
    tooltip=[
        alt.Tooltip('Month_Label:N', title='Month'),
        alt.Tooltip('Metric:N', title='Type'),
        alt.Tooltip('Value:Q', title='Value', format='$,.2f')
    ]
)

combined = line_chart + points
combined


alt.LayerChart(...)

In [35]:
# Visualization: Price difference (how much to adjust)
diff_chart = alt.Chart(pricing_filtered).mark_bar().encode(
    x=alt.X('Month:O', title='Month', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('Price_Difference:Q', title='Price Adjustment Needed ($)'),
    color=alt.condition(
        alt.datum.Price_Difference > 0,
        alt.value('#2ecc71'),  # Green for increase
        alt.value('#e74c3c')   # Red for decrease
    ),
    tooltip=[
        alt.Tooltip('Month_Label:N', title='Month'),
        alt.Tooltip('Price_Difference:Q', title='Adjustment', format='$,.2f'),
        alt.Tooltip('Price_Difference_Percentage:Q', title='% Change', format='.1f')
    ]
).properties(
    title='Required Price Adjustments to Reach Optimal (Positive = Increase, Negative = Decrease)',
    width=700,
    height=400
)

# Add zero line
zero_line = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule(color='black', strokeWidth=1).encode(y='y:Q')

adjustment_chart = diff_chart + zero_line
adjustment_chart


alt.LayerChart(...)

In [36]:
# Export results
pricing_analysis.to_csv("Byte_Datasets/optimal_pricing_lerner_index.csv", index=False)
print("\n✓ Results exported to: Byte_Datasets/optimal_pricing_lerner_index.csv")



✓ Results exported to: Byte_Datasets/optimal_pricing_lerner_index.csv
